In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

# 0 - Read prepped cohorts data

- Read in prepared data and output

In [ ]:
results <- fread(paste0(SHARE_DIR, "2b_go.csv")) 
fisher_base <- fread(paste0(SHARE_DIR, "fisher_base.csv"))

- Quick plot

In [ ]:
p_inv <- function(n) (1-(.05)^(1/n))
    

In [ ]:
go <- data.frame( events = seq(1550), p_response_lt = p_inv(seq(1550)))

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)

In [ ]:
events_with_only_non_response <- results %>% fi(e_r == 0, type == "univariate") %>% se(cohortGo, feature, e_nr)

In [ ]:
events_with_only_non_response %>% ar(desc(e_nr))

In [ ]:
share <- 
ggplot(go %>% fi(p_response_lt < .32), aes(x = events, y = p_response_lt)) + 
 scale_x_log10(limits = c(7, 1550), breaks = c(9, 22, 29, 41, 59, 149, 299, 1550)) + 
 scale_y_continuous(breaks = c(.01, .02, .05, .07, .1, .12, .28), 
                   labels = c("<1%", "<2%", "<5%", "<7%", "<10%", "<12%", "<28%")) +
 geom_point() +
 geom_line() + 
 go_theme + 
 labs( x = "Number of Biomarker Events with No Response", 
       y = "Response Probability < threshold",
       title = "Cohort Sizes and Non-Response thresholds") + 

  ### Most univariate marker non-responder in Hartwig
  geom_segment(aes(x = 0, xend = 22, y = .12, yend = .12), linetype = "dashed", color = "red") +
  geom_segment(aes(x = 22, xend = 22, y = 0, yend = .12), linetype = "dashed", color = "red") + 

  ### 
  geom_segment(aes(x = 0, xend = 9, y = .28, yend = .28), linetype = "dashed", color = "red", alpha = .03) +
  geom_segment(aes(x = 9, xend = 9, y = 0, yend = .28), linetype = "dashed", color = "red", alpha = .3) +

  ### 
  geom_segment(aes(x = 0, xend = 29, y = .1, yend = .1,alpha = .3), linetype = "dashed", color = "grey", alpha = .03) +
  geom_segment(aes(x = 29, xend = 29, y = 0, yend = .1), linetype = "dashed", color = "grey", alpha = .3) +

  ### 
  geom_segment(aes(x = 0, xend = 41, y = .07, yend = .07), linetype = "dashed", color = "red", alpha = .03) +
  geom_segment(aes(x = 41, xend = 41, y = 0, yend = .07), linetype = "dashed", color = "red", alpha = .3) +

  ### 
  geom_segment(aes(x = 0, xend = 59, y = .05, yend = .05), linetype = "dashed", color = "grey", alpha = .3) +
  geom_segment(aes(x = 59, xend = 59, y = 0, yend = .05), linetype = "dashed", color = "grey", alpha = .3) + 

  ###
  geom_segment(aes(x = 0, xend = 149, y = .02, yend = .02), linetype = "dashed", color = "grey", alpha = .3) +
  geom_segment(aes(x = 149, xend = 149, y = 0, yend = .02), linetype = "dashed", color = "grey", alpha = .3) + 

  ###
  geom_segment(aes(x = 0, xend = 299, y = .01, yend = .01), linetype = "dashed", color = "grey", alpha = .3) +
  geom_segment(aes(x = 299, xend = 299, y = 0, yend = .01), linetype = "dashed", color = "grey", alpha = .3) 

In [ ]:
getwd()

In [ ]:
ggsave( "size_never_response.png", plot = share, width = 6, height = 5)#share

- Collect top independent features

In [ ]:
results <- 
results %>% mu( p_bh = p.adjust(fisher_pval, method = "BH"), p_by = p.adjust(fisher_pval, method = "BY")) %>%
 gb(cohortGo) %>% mu(max_patients = max(total_patients))

In [ ]:
dim(results)

In [ ]:
tots <- 
results %>% 
 gb(cohortGo, max_patients, type) %>% 
 su( p_sig = sum(fisher_pval < .05), 
     p_bh_sig = sum(p_bh < .05),
     p_by_sig = sum(p_by < .05), 
     tests = n(), 
     type1 = p_sig/tests, 
     lt_05 = type1 < .05) %>% 
 ug() %>% 
 ar(p_bh_sig) 

In [ ]:
results %>% 
 fi(e_nr > 10, e_r < 3, type != "univariate" ) %>% 
 gb(cohortGo, group) %>% 
 mu(rk = row_number(fisher_pval)) %>% 
 ug() %>% 
 fi(rk <= 1) %>% 
 ar(group, fisher_pval) %>% 
 se(group, cohortGo, feature, e_nr, e_r, ne_nr, ne_r, fisher_pval, p_bh, p_by) %>% 
 fi(group == "type") #%>% 
# fi(grepl("driver", feature))

- Highlighters

In [ ]:
highlighters_univariate <-
c(
 "Pan-Cancer ## Pazopanib" = "rna_geneset_KEGG_PHENYLALANINE_METABOLISM_lt25", 
 "Prostate ## Abiraterone" = "rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt25", 
 "Breast ER+/HER- ## Capecitabine" = "cn_simple_chr21_p_gt75",
 "Pan-Cancer ## Nivolumab" = "rna_geneset_gene_set_t_cell_gep_6_lt25", 
 "Lung NSCLC ## Anti-PD-1" = "neo_ct_lt75", 
 "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50", 
 "Pan-Cancer ## Aromatase inhibitor" = "rna_geneset_HALLMARK_ANDROGEN_RESPONSE_lt50", 
 "Pan-Cancer ## Anti-EGFR" = "drivers_pathway_perplexity_total_gt75",
 "Colorectum ## Chemotherapy" = "hotspot_KRAS_position_25398284", 
 "Skin Melanoma ## Immunotherapy" = "drivers_pathway_IMMUNE_EVASION_gt0", 
 "Breast Triple Negative ## Chemotherapy" = "rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50",
 "Lung NSCLC ## Chemotherapy" = "drivers_pathway_NRF2_gt0"    
)

- Interation Event to Highlight

In [ ]:
highlighters_combo <- 
c( "Pan-Cancer ## Alkaloid ## Platinum" = "cn_simple_chr11_q_gt75_or_purity_msIndelsPerMb_gt75", 
   "Prostate ## Anti-AR" = "chord_BRCA1_gt0_or_rna_geneset_KEGG_COMPLEMENT_AND_COAGULATION_CASCADES_gt25", 
   "Colon ## Chemotherapy" = "hotspot_KRAS_position_25398284_and_purity_tmbPerMb_gt50", 
   "Pan-Cancer ## Multikinase inhibitor" = "cn_simple_chr17_p_lt75_and_rna_geneset_KEGG_PROPANOATE_METABOLISM_lt25", 
   "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_OXIDATIVE_PHOSPHORYLATION_lt75_and_rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50",
   "Lung NSCLC ## Anti-PD-1" = "cn_simple_chr3_p_lt75_and_neo_ct_lt75", 
   "Breast Triple Negative ## Chemotherapy" = "cn_simple_chr22_q_gt25_and_rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50"
 )

In [ ]:
now <- c()
for( i in names(highlighters_univariate)){
    now <- c(now, paste0(i, "--", highlighters_univariate[i]))
}

In [ ]:
map <- c("e_nr" = "Event + No DCB", "e_r" = "Event + DCB", "ne_nr" = "No Event + No DCB", "ne_r" = "No Event + DCB")

In [ ]:
tmp <- 
results %>% 
 se(cohortGo, feature, e_nr, ne_nr, e_r, ne_r, p_bh, group) %>% 
 ga(event, ct, -cohortGo, -feature, -p_bh, -group) %>% 
 mu(event2 = factor(map[event], levels = unname(map)),  event = !m("No Event", event2), dcb = !m("No DCB", event2)) %>% 
 rw() %>% mu(then = paste0(cohortGo, "--", feature)) %>% ug() %>% 
 #fi( (cohortGo %in% names(highlighters)) & (feature %in% unlist(highlighters))) %>%
 fi( then %in% now) %>%
 #fi( then %in% now2) %>%
 gb(cohortGo, feature) %>% 
 mu(nr = max(ct*(as.numeric(event2 == "Event + No DCB")))) %>% 
 ug()

# Figures preparation

In [ ]:
alphas <- c("FALSE" = .4, "TRUE" = 1)
colors <- c("FALSE" = "grey", "TRUE" = "black")
response <- c("FALSE" = "#e52f28", "TRUE" = "#7AABD3")

In [ ]:
options(repr.plot.width =13, repr.plot.height = 4)

In [ ]:
plotter <- function( df, label = "Treatment Examples\n", theme = go_theme, ylab_pt = 30 ){
 
 ggplot(data = df, aes(x = event2, y = ct, alpha = event, fill = dcb, color = event)) + 
  facet_wrap(~fct_reorder(cohortGo, p_bh, .fun = mean) + str_wrap(gsub("_", " ", gsub("rna_geneset_", "",feature)), width = 45) , ncol = 4) +    
  geom_bar(stat = "identity") + 
  scale_fill_manual( values = response) +  
  scale_alpha_manual( values = alphas) + 
  scale_color_manual( values = colors) + 
  geom_text(aes(label = ct), vjust = -0.5, color = "black", size = 3) + 
  scale_y_continuous(
    name = "Number of Patients\n",
    sec.axis = sec_axis(~ ., name = label), 
    expand = expansion(mult = c(0.1, 0.2))
  ) + 
  geom_text(aes(x = 1.3, y = ylab_pt, label = paste0("Adjusted p-value:\n", formatC(p_bh, format = "e", digits = 1))), vjust = -0.5, color = "black", size = 3) + 
  labs(title = "Highlight Individual Molecular Events with Rare Responders", x = "Event Group") +
  theme
}

In [ ]:
top_theme <-
go_theme + 
 theme(strip.background = element_rect(fill = "white", color = "black"), 
       legend.position = "none", 
       axis.title.x = element_blank(),
       axis.text.x = element_blank(),
       axis.ticks.x = element_blank())

In [ ]:
mid_theme <-top_theme + theme(plot.title = element_blank())

In [ ]:
bottom_theme <-
go_theme + 
 theme(strip.background = element_rect(fill = "white", color = "black"), 
       legend.position = "none", 
       plot.title = element_blank())

In [ ]:
top <- plotter(tmp %>% fi(group == "treatment"), theme = top_theme, label = "Treatment\n",  ylab_pt = 45)
mid <- plotter(tmp %>% fi(group == "mechanism"), theme = mid_theme, label = "Mechanism\n", ylab_pt = 28)
bottom <- plotter(tmp %>% fi(group == "type"), theme = bottom_theme, label = "Type\n", ylab_pt = 85)

In [ ]:
options(repr.plot.width =14, repr.plot.height = 9)
library(patchwork)

In [ ]:
univariate <- (top / mid / bottom ) + plot_layout(heights = c(4, 4, 5)) 

In [ ]:
getwd()

In [ ]:
univariate
ggsave( "univariate_examples.png", plot = univariate, width = 14, height = 9)

# Generally Bad Prognostic Features 